# Оценка модели
Строим модель энергопотребления здания по среднему значению. Оцениваем эффективность модели через метрику
$$\LARGE RMSLE = \sqrt{\frac{\sum_{i=1}^{n}(log(p_i+1)-log(a_i+1))^2}{n}},$$
где:
- $n - число\; наблюдений;$
- $log\;- натуральный\; логорифм;$
- $p_i\;- вычисленное\; значение\; метрики;$
- $a_i\;- заданное\; значение\; метрики.$

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Загрузка данных. Выкунуть пустые дни по "meter_reading" и выделить час в отдельную колонку.
**meter_reading** - потребление энергии в кВтч (или эквивалент)

In [2]:
energy = pd.read_csv("../data/train.csv")
energy = energy[energy["meter_reading"]>0]
energy["timestamp"] = pd.to_datetime(energy["timestamp"])
energy["hour"] = energy["timestamp"].dt.hour
print(energy.head())

      building_id  meter           timestamp  meter_reading  hour
704             0      0 2016-01-30 08:00:00        43.6839     8
725             0      0 2016-01-31 05:00:00        37.5408     5
737             0      0 2016-01-31 17:00:00        52.5571    17
2366            0      0 2016-04-08 14:00:00        59.3827    14
2923            0      0 2016-05-01 19:00:00       448.0000    19


# Разделить данные на обучение и проверку
**20%** - на проверку
**80%** - на обучение

In [3]:
energy_train, energy_test = train_test_split(energy, test_size=0.2)
print(energy_train.count())

building_id      4328
meter            4328
timestamp        4328
meter_reading    4328
hour             4328
dtype: int64


## Среднее и медианное значение потребления энергии по часам

In [4]:
energy_train_hours = energy_train.groupby("hour")
energy_meter_reading = energy_train_hours["meter_reading"]
energy_train_avg = pd.DataFrame(
    {
        "Среднее": energy_meter_reading.mean(),
        "Медиана": energy_meter_reading.median(),
    }
)
print(energy_train_avg)

         Среднее   Медиана
hour                      
0     240.947455  241.6260
1     238.392139  240.9440
2     237.314296  240.9440
3     236.428086  241.6260
4     240.749304  245.7215
5     239.053290  245.7220
6     238.192342  245.7220
7     238.979083  246.4040
8     242.301786  238.5550
9     235.235792  234.1180
10    236.683173  238.8960
11    233.212067  241.6260
12    234.180228  242.3090
13    237.338659  243.6740
14    235.736723  242.3090
15    238.664932  243.3325
16    234.970421  242.3090
17    235.199551  242.3090
18    236.712641  239.5790
19    238.461354  237.8725
20    233.699342  236.1660
21    239.307959  238.2140
22    235.822716  236.1660
23    240.226683  239.9200


## Проверка модели

In [8]:
def model(x):
    meter_reading_log = np.log(x.meter_reading + 1)
    meter_reading_mean = np.log(energy_train_avg["Среднее"][x.hour] + 1)
    meter_reading_median = np.log(energy_train_avg["Медиана"][x.hour] + 1)
    
    x["meter_reading_mean_q"] = (meter_reading_log - meter_reading_mean)**2
    x["meter_reading_median_q"] = (meter_reading_log - meter_reading_median)**2
    x["meter_reading_zero_q"] = meter_reading_log**2

    return x

energy_test = energy_test.apply(model, axis=1, result_type="expand")
print(energy_test.head())

      building_id  meter           timestamp  meter_reading  hour  \
7035            0      0 2016-10-20 03:00:00        230.023     3   
4317            0      0 2016-06-28 21:00:00        233.436    21   
8537            0      0 2016-12-21 17:00:00        233.436    17   
7905            0      0 2016-11-25 09:00:00        208.181     9   
5389            0      0 2016-08-12 13:00:00        307.152    13   

      meter_reading_mean_q  meter_reading_median_q  meter_reading_zero_q  
7035              0.000748                0.002401             29.620994  
4317              0.000612                0.000407             29.780842  
8537              0.000056                0.001380             29.780842  
7905              0.014794                0.013663             28.549785  
5389              0.065998                0.053207             32.839698  


In [9]:
len_ = len(energy_test)
energy_test_median_rmsle = np.sqrt(energy_test["meter_reading_median_q"].sum() / len_)
energy_test_mean_rmsle = np.sqrt(energy_test["meter_reading_mean_q"].sum() / len_)
energy_test_zero_rmsle = np.sqrt(energy_test["meter_reading_zero_q"].sum() / len_)

print(f"Качество медианы: {energy_test_median_rmsle}")
print(f"Качество среднего: {energy_test_mean_rmsle}")
print(f"Качество нуля: {energy_test_zero_rmsle}")

Качество медианы: 0.2532163249707196
Качество среднего: 0.25144727480521406
Качество нуля: 5.459743329271214
